In [1]:
import os
import requests
from logging import Logger
from PIL import Image
import base64
from io import BytesIO
import streamlit as st
import pandas as pd
from ai_eval.services.file_gcp import PDFService
from ai_eval.services.logger import LoggerFactory
from ai_eval.utils import utils
from ai_eval.config import global_config as glob
from ai_eval.utils.utils import list_objects
from ai_eval.resources.predictor import Predictor

2025-04-24 00:07:14,101 - ai_eval.services.file_gcp - INFO - Logger created for module: ai_eval.services.file_gcp using log level: DEBUG
/Users/avosseler/Github/Team/ai-assistant-eval/.venv/lib/python3.12/site-packages/deepeval/__init__.py:54: UserWarning: You are using deepeval version 2.7.1, however version 2.7.6 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(
2025-04-24 00:07:31,103 - ai_eval.services.file - INFO - Logger created for module: ai_eval.services.file using log level: DEBUG
2025-04-24 00:07:31,174 - ai_eval.resources.llm_aaj - INFO - Logger created for module: ai_eval.resources.llm_aaj using log level: DEBUG
2025-04-24 00:07:31,257 - ai_eval.resources.get_models - INFO - Logger created for module: ai_eval.resources.get_models using log level: DEBUG
2025-04-24 00:07:31,325 - ai_eval.services.opik - INFO - Logger created for module: ai_eval.services.opik using log level: DEBUG
2025-04-24 00:07:31,354 - ai_eval.

In [2]:
# from google.cloud import storage

# storage_client = storage.Client(project=glob.GCP_PROJECT)

# blobs = storage_client.list_blobs(glob.GCP_CS_BUCKET)
# [blob.name for blob in blobs]

In [3]:
# from google.cloud import firestore

# # Initialize Firestore client
# firestore_client = firestore.Client(project=glob.GCP_PROJECT, database=glob.GCP_FIRESTORE)

# # Set a timeout for the operation
# timeout = 600  # 600 seconds (10 minutes)

# # Retrieve collections with the specified timeout
# collections = list(firestore_client.collections(timeout=timeout))
# print([collection.id for collection in collections])

In [3]:
out = list_objects(bucket_name=glob.GCP_CS_BUCKET)

In [2]:
from langchain.document_loaders import PyPDFLoader

filename = "20240731_Nemetschek SE_Mitarbeiterhandbuch_Employee Handbook.pdf"

loader = PyPDFLoader(f"{glob.DATA_PKG_DIR}/{filename}")

raw_data = loader.load()

In [3]:
from ai_eval.resources.synthesizer import Synthesizer
from ai_eval.resources.get_models import InitModels

models = InitModels()

rag_generator = models.rag_generator

synth = Synthesizer(
    qa_generator=models.qa_generator,
    eval_model=models.eval_model,
    blob_path="raw_documents/20240731_Nemetschek SE_Mitarbeiterhandbuch_Employee Handbook.pdf",
    verbose=True,
)

out = synth.create_qa_data(
    n_generations=3,
)

2025-04-24 00:07:32,169 - ai_eval.resources.preprocessor - INFO - Logger created for module: ai_eval.resources.preprocessor using log level: DEBUG
2025-04-24 00:07:32,480 - ai_eval.services.database - INFO - Logger created for module: ai_eval.services.database using log level: DEBUG
2025-04-24 00:07:32,503 - ai_eval.resources.synthesizer - INFO - Logger created for module: ai_eval.resources.synthesizer using log level: DEBUG
2025-04-24 00:07:32,504 - ai_eval.resources.get_models - INFO - Fetching google models: ['qa_generator', 'rag_generator', 'eval_model']
2025-04-24 00:07:32,504 - ai_eval.resources.get_models - INFO - Models loaded successfully
2025-04-24 00:07:32,504 - ai_eval.resources.get_models - INFO - QA Generator: project='nm-sbox-neme-dev-prj-01' max_retries=2 model_name='gemini-2.0-flash-001' full_model_name='projects/nm-sbox-neme-dev-prj-01/locations/us-central1/publishers/google/models/gemini-2.0-flash-001' client_options=ClientOptions: {'api_endpoint': 'us-central1-aipla


Starting Test Dataset Creation...


2025-04-24 00:07:35,249 - ai_eval.services.database - INFO - 
Database: projects/nm-sbox-neme-dev-prj-01/databases/ai-assistant-evaluation
2025-04-24 00:07:35,250 - ai_eval.services.database - INFO - Initialized Firestore collection: Synthesis-Requests

2025-04-24 00:07:35,271 - ai_eval.services.opik - INFO - Opik client initialized with project 'A.I. Assistant Evaluation 🤖'
2025-04-24 00:07:35,271 - ai_eval.resources.synthesizer - INFO - Synthesizer initialized.
2025-04-24 00:07:35,271 - ai_eval.resources.synthesizer - INFO - Creating 3 Q/A test cases 🙃 ...
2025-04-24 00:07:35,332 - ai_eval.resources.synthesizer - INFO - Document for request 9c10b532-bd17-41fe-905b-536491a1080c created in Firestore.
2025-04-24 00:07:35,333 - ai_eval.resources.synthesizer - INFO - Loading and chunking documents...


Finished '_get_all_document_ids' in 0.3099 secs


2025-04-24 00:07:36,743 - ai_eval.resources.preprocessor - INFO - Processed raw_documents/20240731_Nemetschek SE_Mitarbeiterhandbuch_Employee Handbook.pdf
2025-04-24 00:07:36,744 - ai_eval.resources.llm_aaj - INFO - Generating 3 QA couples...
2025-04-24 00:07:36,745 - ai_eval.resources.llm_aaj - INFO - Sampling 3 contexts without replacement


Finished 'fetch_documents' in 1.4103 secs
Finished 'chunk_documents' in 0.0008 secs


  0%|          | 0/3 [00:00<?, ?it/s]

Finished 'generate_qa_couples' in 4.2033 secs
Rating each QA couple according to groundedness, question relevancy and answer faithfulness...


Evaluating QA couples:   0%|          | 0/3 [00:00<?, ?it/s]


When must you report your inability to work?
Immediately (usually by 9 am).

Where do I get an access card?
You will receive your access card from reception.

Where can Nemetschek employees register for discounted purchasing opportunities?
https://nemetschek.vorteile.net/de/login and https://nemetschekgroup.mitarbeiterangebote.de
Filtered QA pairs: 3 out of 3


In [4]:
synth.upload_to_opik(
    dataset_name="My Test Dataset",
)

2025-04-22 22:04:23,206 - ai_eval.services.opik - INFO - Creating/Getting Opik dataset 'My Test Dataset'
OPIK: Created a "My Test Dataset" dataset at https://www.comet.com/opik/api/v1/session/redirect/datasets/?dataset_id=01965f1a-08b3-74a1-b8d2-8f75b89a86b2&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
2025-04-22 22:04:24,001 - ai_eval.services.opik - INFO - Inserting items from DataFrame into Opik dataset 'My Test Dataset'
2025-04-22 22:04:24,146 - ai_eval.resources.synthesizer - INFO - Dataset uploaded to Opik: My Test Dataset 📁


In [9]:
synth.save_data(file_name="generated_test_data.csv")

2025-04-08 19:11:50,302 - ai_eval.services.file_gcp - INFO - Writing CSV file to path: raw_documents/generated_test_data.csv
2025-04-08 19:11:50,627 - ai_eval.services.file_gcp - INFO - CSV Service Output to File: raw_documents/generated_test_data.csv
2025-04-08 19:11:50,627 - ai_eval.services.file_gcp - INFO - Successfully wrote CSV file to path: raw_documents/generated_test_data.csv


In [11]:
data = synth.generated_qa
data.head()

,question,answer,context,groundedness_score,groundedness_eval,question_relevancy_score,question_relevancy_eval,faithfulness_score,faithfulness_eval
0,What should be done with open files when leavi...,Open files must be closed or the screen saver ...,17 \nwird. Der Sicherung der Zukunft unseres U...,5,The context explicitly states that when leavin...,5,This question is highly relevant to data secur...,5,The answer is completely faithful to the conte...
1,What language courses does Nemetschek offer it...,English and German,15 \n6. Personalentwicklung & Zielver-\neinbar...,5,The context explicitly states that Nemetschek ...,5,Knowing what language courses are available is...,5,The answer accurately reflects the information...
2,Where is the data garbage can located?,Corridor of the 3rd floor.,"secrets of the employer, companies economicall...",5,The context explicitly states that the data ga...,4,This question is likely referring to a data re...,5,The answer is completely faithful to the conte...
3,What is the weekly target working time for ful...,39 hours,2 \n2. Corporate Identity 2. Corporate Identit...,5,The context explicitly states the weekly targe...,5,Knowing the standard weekly working hours is f...,5,The answer accurately reflects the information...
4,How many days can you be released from work to...,Up to 5 days.,"5 \nverpflichtet, eine neue ärztliche Beschein...",5,The context explicitly states that you have th...,5,This question is highly relevant to employees ...,5,The answer is completely faithful to the conte...


In [2]:
filename: str = "generated_test_data.csv"

In [ ]:
from langchain.docstore.document import Document

docs_processed = [
    Document(page_content=context, metadata={"source": filename})
    for context in data["context"].tolist()
]

In [ ]:
out[0]

{'context': 'ment for the glasses is financed by your private \nfunds. \n \nThe frequency of reimbursement depends on the \nindividual change in the eyes and a renewed con-\nfirmation by the company doctor or by an oph-\nthalmologist if a supplementary examination has \nbeen carried out on the instructions of the com-\npany doctor. \n \nVerbilligte Einkaufsmöglichkeiten \nMitarbeitende der Nemetschek Group erhalten \nüber zwei Portale die Möglichkeit bei zahlreichen \nAnbietern spezielle Angebote und exklusive Ver-\ngünstigungen zu erwerben.  Auf den folgenden \nSeiten kannst du dich mit deiner Nemetschek E-\nMail-Adresse registrieren:  \nhttps://nemetschek.vorteile.net/de/login  \nhttps://nemetschekgroup.mitarbeiterangebote.de  \n \nDiscounted purchasing opportunities \nNemetschek Group employees have the oppor-\ntunity to purchase special offers and exclusive \ndiscounts from numerous providers via two por-\ntals. Here you can register with your Nemetschek \ne-mail address: \nhttps:/

In [4]:
from ai_eval.resources.llm_aaj import (
    build_local_vectorstore,
    answer_with_rag,
    # answer_with_rag_async,
)

pred = Predictor()

# docs_processed = pred.load_test_data()
#vectorstore = build_local_vectorstore(docs_processed)

2025-04-24 00:07:51,807 - ai_eval.services.opik - INFO - Opik client initialized with project 'A.I. Assistant Evaluation 🤖'


In [4]:
sample_queries = data["question"].tolist()
expected_responses = data["answer"].tolist()
input_contexts = data["context"].tolist()

print(sample_queries)
print(expected_responses)
print(input_contexts)

NameError: name 'data' is not defined

In [29]:
X1 = iter(sample_queries)
X2 = iter(input_contexts)
Y = iter(expected_responses)

test_cases = []

Increment examples:

In [30]:
query = next(X1)
ground_truth_context = next(X2)
ground_truth_answer = next(Y)

print(ground_truth_context[:150])
print(query)
print(ground_truth_answer)

ment for the glasses is financed by your private 
funds. 
 
The frequency of reimbursement depends on the 
individual change in the eyes and a renewed
How are glasses financed?
By private funds.


In [31]:
predicted_answer, predicted_context = answer_with_rag(
    question=query, llm=rag_generator, vectorstore=vectorstore
)

from deepeval.test_case import LLMTestCase
from deepeval.dataset import EvaluationDataset

test_case = LLMTestCase(
    input=query,
    actual_output=predicted_answer.content,
    expected_output=ground_truth_answer,
    retrieval_context=predicted_context,
)

test_cases.append(test_case)

print(f"{len(test_cases)} test cases created")

#print(f"Retrieved {len(relevant_docs)} contexts")
print(f"Relevant (NN-) context:\n{predicted_context[0][:200]}")
print(f"Ground truth: {ground_truth_context[:200]}")
print(f"Expected: {ground_truth_answer}")
print(f"RAG: {predicted_answer.content}")

EvaluationDataset(test_cases=test_cases)

1 test cases created
Relevant (NN-) context:
ment for the glasses is financed by your private 
funds. 
 
The frequency of reimbursement depends on the 
individual change in the eyes and a renewed con-
firmation by the company doctor or by an oph
Ground truth: ment for the glasses is financed by your private 
funds. 
 
The frequency of reimbursement depends on the 
individual change in the eyes and a renewed con-
firmation by the company doctor or by an oph
Expected: By private funds.
RAG: The glasses are financed by your private funds. (Document No. 0)


EvaluationDataset(test_cases=[LLMTestCase(input='How are glasses financed?', actual_output='The glasses are financed by your private funds. (Document No. 0)', expected_output='By private funds.', context=None, retrieval_context=['ment for the glasses is financed by your private \nfunds. \n \nThe frequency of reimbursement depends on the \nindividual change in the eyes and a renewed con-\nfirmation by the company doctor or by an oph-\nthalmologist if a supplementary examination has \nbeen carried out on the instructions of the com-\npany doctor. \n \nVerbilligte Einkaufsmöglichkeiten \nMitarbeitende der Nemetschek Group erhalten \nüber zwei Portale die Möglichkeit bei zahlreichen \nAnbietern spezielle Angebote und exklusive Ver-\ngünstigungen zu erwerben.  Auf den folgenden \nSeiten kannst du dich mit deiner Nemetschek E-\nMail-Adresse registrieren:  \nhttps://nemetschek.vorteile.net/de/login  \nhttps://nemetschekgroup.mitarbeiterangebote.de  \n \nDiscounted purchasing opportunities \nN

In [5]:
evaluation_dataset = pred.predict()

print(evaluation_dataset)

2025-04-24 00:07:51,821 - ai_eval.resources.predictor - INFO - Loading prepared test cases from cloud storage...
2025-04-24 00:07:52,155 - ai_eval.services.file_gcp - INFO - Reading CSV file from path: raw_documents/generated_test_data.csv
2025-04-24 00:07:52,432 - ai_eval.services.file_gcp - INFO - CSV Service Read from File: raw_documents/generated_test_data.csv
2025-04-24 00:07:52,432 - ai_eval.services.file_gcp - INFO - Successfully read CSV file from path: raw_documents/generated_test_data.csv
2025-04-24 00:07:52,433 - ai_eval.resources.predictor - INFO - Loaded 2 documents.
2025-04-24 00:07:52,433 - ai_eval.resources.llm_aaj - INFO - Processing 2 valid out of 2 documents for vectorstore.
2025-04-24 00:07:53,576 - ai_eval.resources.llm_aaj - INFO - Building FAISS local vectorstore...
2025-04-24 00:07:54,619 - ai_eval.resources.llm_aaj - INFO - Vectorstore built.


Finished 'build_local_vectorstore' in 2.1865 secs


Building test dataset:   0%|          | 0/2 [00:00<?, ?it/s]

2025-04-24 00:07:59,203 - ai_eval.resources.predictor - INFO - Created 2 test cases.


                                   question          answer  \
0                        What date is Holi?        March 14   
1  When is Telangana Formation Day in 2025?  June 2, Monday   

                                             context  groundedness_score  \
0  19 20 21 22 23 24 25 16 17 18 19 20 21 22 21 2...                   5   
1  Holiday Calendar 2025 \n \nJanuary 1, Wednesda...                   5   

                                   groundedness_eval  \
0  The context clearly states that Holi is on Mar...   
1  The context clearly states that Telangana Form...   

   question_relevancy_score  \
0                         2   
1                         1   

                             question_relevancy_eval  faithfulness_score  \
0  Holi is a Hindu festival. While Nemetschek is ...                   5   
1  This question is not relevant to the employees...                   5   

                                   faithfulness_eval  \
0  The answer is completely faith

In [17]:
pred.evaluation_dataset

EvaluationDataset(test_cases=[LLMTestCase(input='When is Telangana Formation Day in 2025?', actual_output='Telangana Formation Day is on June 2, 2025 (Document No. 0).\n', expected_output='June 2, Monday', context=['Holiday Calendar 2025 \n \nJanuary 1, Wednesday \nNew Year’s Day \n \nJanuary 14, Tuesday \nMakara Sankranti \n \nMarch 31, Monday \nIdul Fitr Holiday \n \nMay 1, Thursday \nMay day \n \nJune 2, Monday \nTelangana Formation Day \n \nAugust 15, Friday \nIndependence Day \n \nAugust 27, Wednesday \nVinayaka Chaturthi \n \nOctober 2, Thursday \nGandhi Jayanti / Dusshera \n \nOctober 20, Monday \nDeepavali \n \nDecember 25, Thursday \nChristmas Day \nJanuary \nSu Mo Tu We Th Fr Sa \n   1 2 3 4 \n5 6 7 8 9 10 11 \n12 13 14 15 16 17 18 \n19 20 21 22 23 24 25 \n26 27 28 29 30 31  \n \nFebruary \nSu Mo Tu We Th Fr Sa \n      1 \n2 3 4 5 6 7 8 \n9 10 11 12 13 14 15 \n16 17 18 19 20 21 22 \n23 24 25 26 27 28  \n \nMarch \nSu Mo Tu We Th Fr Sa \n      1 \n2 3 4 5 6 7 8 \n9 10 11 12 13

In [14]:
evaluation_dataset

,question,answer,context,groundedness_score,groundedness_eval,question_relevancy_score,question_relevancy_eval,faithfulness_score,faithfulness_eval,predicted_answer,predicted_context
0,When is Telangana Formation Day in 2025?,"June 2, Monday","Holiday Calendar 2025 \n \nJanuary 1, Wednesda...",5,The context explicitly states that Telangana F...,1,This question is not relevant to the employees...,5,The answer accurately reflects the information...,"Telangana Formation Day is on June 2, 2025 (Do...","Holiday Calendar 2025 \n \nJanuary 1, Wednesda..."
1,What date is Holi?,March 14,19 20 21 22 23 24 25 16 17 18 19 20 21 22 21 2...,5,The context clearly states that Holi is on Mar...,1,"The question ""What date is Holi?"" is not direc...",5,The answer is completely faithful to the conte...,March 14 (Document 0)\n,19 20 21 22 23 24 25 16 17 18 19 20 21 22 21 2...


In [48]:
evaluation_dataset.save_as(
    file_type="json",
    directory=os.path.join(glob.DATA_PKG_DIR, "tmp"),
    include_test_cases=True
)

Evaluation dataset saved at /Users/avosseler/Github/Team/ai-assistant-eval/data/tmp/20250408_163952.json!


'/Users/avosseler/Github/Team/ai-assistant-eval/data/tmp/20250408_163952.json'

In [3]:
local_tmp_json = os.listdir(os.path.join(glob.DATA_PKG_DIR, "tmp"))[0]
print(local_tmp_json)

20250408_163952.json


In [ ]:
from ai_eval.services.file import JSONService

js = JSONService(path=local_tmp_json, root_path=os.path.join(glob.DATA_PKG_DIR, "tmp"))

my_set = js.doRead()
my_set

2025-04-08 16:49:34,123 - ai_eval.services.file - INFO - Read: /Users/avosseler/Github/Team/ai-assistant-eval/data/tmp/20250408_163952.json


[{'input': 'How are glasses financed?',
  'actual_output': 'The glasses are financed by your private funds. (Document No. 0)',
  'expected_output': 'By private funds.',
  'retrieval_context': ['ment for the glasses is financed by your private \nfunds. \n \nThe frequency of reimbursement depends on the \nindividual change in the eyes and a renewed con-\nfirmation by the company doctor or by an oph-\nthalmologist if a supplementary examination has \nbeen carried out on the instructions of the com-\npany doctor. \n \nVerbilligte Einkaufsmöglichkeiten \nMitarbeitende der Nemetschek Group erhalten \nüber zwei Portale die Möglichkeit bei zahlreichen \nAnbietern spezielle Angebote und exklusive Ver-\ngünstigungen zu erwerben.  Auf den folgenden \nSeiten kannst du dich mit deiner Nemetschek E-\nMail-Adresse registrieren:  \nhttps://nemetschek.vorteile.net/de/login  \nhttps://nemetschekgroup.mitarbeiterangebote.de  \n \nDiscounted purchasing opportunities \nNemetschek Group employees have the o

In [5]:
from ai_eval.services.file_gcp import JSONService as JSONService_GCP

json_service = JSONService_GCP(
    root_path="raw_documents",
    path="test.json",
)
json_service.doWrite(X=my_set)

2025-04-08 16:49:42,088 - ai_eval.services.file_gcp - INFO - Writing JSON file to path: raw_documents/test.json
2025-04-08 16:49:42,494 - ai_eval.services.file_gcp - INFO - Successfully wrote JSON file to path: raw_documents/test.json


In [6]:
os.remove(os.path.join(glob.DATA_PKG_DIR, "tmp", local_tmp_json))

In [8]:
from importlib import reload

reload(utils);

In [13]:
from ai_eval.utils.utils import LangChainDeepEvalModel
from deepeval import evaluate
from deepeval.metrics import (
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric,
    AnswerRelevancyMetric, 
    FaithfulnessMetric
)
from ai_eval.resources.get_models import InitModels

models = InitModels()

# Wrap the LangChain model with your custom DeepEval class
deepeval_model = LangChainDeepEvalModel(model=models.eval_model)


metric1 = AnswerRelevancyMetric(model=deepeval_model, threshold=0.5, include_reason=True)

metric2 = FaithfulnessMetric(
    threshold=0.7,
    model=deepeval_model,
    include_reason=True
)

metric3 = ContextualRecallMetric(
    threshold=0.7,
    model=deepeval_model,
    include_reason=True
)

metric4 = ContextualPrecisionMetric(
        threshold=0.7,
        model=deepeval_model,
        include_reason=True    
)

metric5 = ContextualRelevancyMetric(
        threshold=0.7,
        model=deepeval_model,
        include_reason=True
)


# evaluation_dataset.evaluate([metric1, metric2, metric3])

results = evaluate(pred.evaluation_dataset, 
         metrics=[metric1, metric2, metric3, metric4], 
         ignore_errors=False,
         write_cache=True  # Ensures results are saved locally
        )

2025-04-23 15:24:00,563 - ai_eval.resources.get_models - INFO - Fetching google models: ['qa_generator', 'rag_generator', 'eval_model']
2025-04-23 15:24:00,564 - ai_eval.resources.get_models - INFO - Models loaded successfully
2025-04-23 15:24:00,565 - ai_eval.resources.get_models - INFO - QA Generator: project='nm-sbox-neme-dev-prj-01' max_retries=2 model_name='gemini-2.0-flash-001' full_model_name='projects/nm-sbox-neme-dev-prj-01/locations/us-central1/publishers/google/models/gemini-2.0-flash-001' client_options=ClientOptions: {'api_endpoint': 'us-central1-aiplatform.googleapis.com', 'client_cert_source': None, 'client_encrypted_cert_source': None, 'quota_project_id': None, 'credentials_file': None, 'scopes': None, 'api_key': None, 'api_audience': None, 'universe_domain': None} default_metadata=() temperature=0.3 model_family=<GoogleModelFamily.GEMINI_ADVANCED: '2'>
2025-04-23 15:24:00,565 - ai_eval.resources.get_models - INFO - RAG Generator: project='nm-sbox-neme-dev-prj-01' max_r

✨ You're running DeepEval's latest Answer Relevancy Metric! (using LangChain Model, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using LangChain Model, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using LangChain Model, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using LangChain Model, strict=False, 
async_mode=True)...

Evaluating 2 test case(s) in parallel: |          |  0% (0/2) [Time Taken: 00:00, ?test case/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1745414640.629829 1741583 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
Evaluating 2 test case(s) in parallel: |██████████|100% (2/2) [Time Taken: 00:04,  2.28s/test case]



Metrics Summary

  - ✅ Answer Relevancy (score: 0.5, threshold: 0.5, strict: False, evaluation model: LangChain Model, reason: The score is 0.50 because the inclusion of 'Doc. 0' which is not a date, significantly detracts from the relevancy of the response., error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: LangChain Model, reason: The score is 1.00 because there are no contradictions, indicating the output is perfectly faithful to the retrieval context! Great job!, error: None)
  - ✅ Contextual Recall (score: 1.0, threshold: 0.7, strict: False, evaluation model: LangChain Model, reason: The score is 1.00 because the 1st sentence in the expected output is fully supported by the 1st node in retrieval context. Great job!, error: None)
  - ✅ Contextual Precision (score: 1.0, threshold: 0.7, strict: False, evaluation model: LangChain Model, reason: The score is 1.00 because the first node in the retrieval contexts is relevant and provides the 

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

In [15]:
results.test_results

[TestResult(name='test_case_1', success=True, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=0.5, reason="The score is 0.50 because the inclusion of 'Doc. 0' which is not a date, significantly detracts from the relevancy of the response.", strict_mode=False, evaluation_model='LangChain Model', error=None, evaluation_cost=None, verbose_logs='Statements:\n[\n    "March 14",\n    "Doc. 0"\n] \n \nVerdicts:\n[\n    {\n        "verdict": "yes",\n        "reason": null\n    },\n    {\n        "verdict": "no",\n        "reason": "Doc. 0 is not a date and is irrelevant to the question."\n    }\n]'), MetricData(name='Faithfulness', threshold=0.7, success=True, score=1.0, reason='The score is 1.00 because there are no contradictions, indicating the output is perfectly faithful to the retrieval context! Great job!', strict_mode=False, evaluation_model='LangChain Model', error=None, evaluation_cost=None, verbose_logs='Truths (limit=None):\n[\n    "The calendar

In [19]:
from opik import track

@track(name="DeepEval Evaluation", project_name=glob.OPIK_PROJECT_NAME)
def run_evaluation():
    # Run DeepEval tests
    results = evaluate(pred.evaluation_dataset, 
         metrics=[metric1, metric2, metric3, metric4], 
         ignore_errors=False,
         write_cache=True  # Ensures results are saved locally
        )
    return results


In [20]:
run_evaluation()

✨ You're running DeepEval's latest Answer Relevancy Metric! (using LangChain Model, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using LangChain Model, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using LangChain Model, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using LangChain Model, strict=False, 
async_mode=True)...

Evaluating 2 test case(s) in parallel: |██████████|100% (2/2) [Time Taken: 00:03,  1.91s/test case]



Metrics Summary

  - ✅ Answer Relevancy (score: 0.5, threshold: 0.5, strict: False, evaluation model: LangChain Model, reason: The score is 0.50 because the inclusion of a document that is explicitly not a date significantly detracts from the overall relevance, even if other parts of the response are relevant., error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: LangChain Model, reason: The score is 1.00 because there are no contradictions, indicating the output aligns perfectly with the provided context! Great job!, error: None)
  - ✅ Contextual Recall (score: 1.0, threshold: 0.7, strict: False, evaluation model: LangChain Model, reason: The score is 1.00 because the 1st node in retrieval context perfectly contains the expected output 'March 14'!, error: None)
  - ✅ Contextual Precision (score: 1.0, threshold: 0.7, strict: False, evaluation model: LangChain Model, reason: The score is 1.00 because the first node in the retrieval contexts is 

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

EvaluationResult(test_results=[TestResult(name='test_case_1', success=True, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=0.5, reason='The score is 0.50 because the inclusion of a document that is explicitly not a date significantly detracts from the overall relevance, even if other parts of the response are relevant.', strict_mode=False, evaluation_model='LangChain Model', error=None, evaluation_cost=None, verbose_logs='Statements:\n[\n    "March 14",\n    "Doc. 0"\n] \n \nVerdicts:\n[\n    {\n        "verdict": "yes",\n        "reason": null\n    },\n    {\n        "verdict": "no",\n        "reason": "Doc. 0 is not a date and therefore irrelevant to the question."\n    }\n]'), MetricData(name='Faithfulness', threshold=0.7, success=True, score=1.0, reason='The score is 1.00 because there are no contradictions, indicating the output aligns perfectly with the provided context! Great job!', strict_mode=False, evaluation_model='LangChain Model', erro

In [24]:
type(metric1)

deepeval.metrics.answer_relevancy.answer_relevancy.AnswerRelevancyMetric

In [9]:
my_test_cases = pred.evaluation_dataset.test_cases

my_test_cases[0].actual_output

'March 14 (Document 0)\n'

In [11]:
test_case = my_test_cases[1]

metric1.measure(test_case)
metric2.measure(test_case)
metric3.measure(test_case)
metric4.measure(test_case)
    
scores = {metric_name:metric.score for metric_name, metric in zip(["Answer Relevancy", "Faithfulness", "Contextual Recall", "Contextual Precision"], [metric1, metric2, metric3, metric4])}

#scores = [metric.score for metric in [metric1, metric2, metric3, metric4]]
print(scores)

Output()

Output()

Output()

Output()

{'Answer Relevancy': 0.5, 'Faithfulness': 1.0, 'Contextual Recall': 1.0, 'Contextual Precision': 1.0}


In [ ]:
from opik import Experiment

experiment = Experiment(name="DeepEval Integration Test")
for item in dataset:
    # Assume you have DeepEval results in `result`
    experiment.log(
        input=item["input"],
        output=your_llm_output,
        expected_output=item["expected_output"],
        metrics={"accuracy": result.score},  # Add other metrics as needed
        context=item.get("context", "")
    )
experiment.end()


In [6]:
from ai_eval.resources.deepeval_scorer import DeepEvalScorer

evaluation_dataset = pred.evaluation_dataset

2025-04-24 00:07:59,270 - ai_eval.resources.deepeval_scorer - INFO - Logger created for module: ai_eval.resources.deepeval_scorer using log level: DEBUG


In [7]:
# Initialize the scorer
scorer = DeepEvalScorer(evaluation_dataset)

2025-04-24 00:07:59,280 - ai_eval.resources.get_models - INFO - Fetching google models: ['qa_generator', 'rag_generator', 'eval_model']
2025-04-24 00:07:59,280 - ai_eval.resources.get_models - INFO - Models loaded successfully
2025-04-24 00:07:59,280 - ai_eval.resources.get_models - INFO - QA Generator: project='nm-sbox-neme-dev-prj-01' max_retries=2 model_name='gemini-2.0-flash-001' full_model_name='projects/nm-sbox-neme-dev-prj-01/locations/us-central1/publishers/google/models/gemini-2.0-flash-001' client_options=ClientOptions: {'api_endpoint': 'us-central1-aiplatform.googleapis.com', 'client_cert_source': None, 'client_encrypted_cert_source': None, 'quota_project_id': None, 'credentials_file': None, 'scopes': None, 'api_key': None, 'api_audience': None, 'universe_domain': None} default_metadata=() temperature=0.3 model_family=<GoogleModelFamily.GEMINI_ADVANCED: '2'>
2025-04-24 00:07:59,281 - ai_eval.resources.get_models - INFO - RAG Generator: project='nm-sbox-neme-dev-prj-01' max_r

In [8]:
# Calculate and log scores to Opik
results = scorer.calculate_scores()
print(results.test_results)

✨ You're running DeepEval's latest Answer Relevancy Metric! (using LangChain Model, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using LangChain Model, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using LangChain Model, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using LangChain Model, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using LangChain Model, strict=False, 
async_mode=True)...

Evaluating 2 test case(s) in parallel: |          |  0% (0/2) [Time Taken: 00:00, ?test case/s]/Users/avosseler/Github/Team/ai-assistant-eval/src/ai_eval/utils/utils.py:181: LangChainDeprecationWarning: The method `BaseChatModel.apredict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~ainvoke` instead.
  res = await chat_model.apredict(prompt)
Evaluating 2 test case(s) in parallel: |██████████|100% (2/2) [Time Taken: 00:18,  9.02s/test case]



Metrics Summary

  - ✅ Answer Relevancy (score: 0.5, threshold: 0.5, strict: False, evaluation model: LangChain Model, reason: The score is 0.50 because the inclusion of irrelevant document numbers detracts from the directness of the answer., error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: LangChain Model, reason: The score is 1.00 because there are no contradictions, indicating the actual output aligns perfectly with the retrieval context! Great job!, error: None)
  - ✅ Contextual Recall (score: 1.0, threshold: 0.7, strict: False, evaluation model: LangChain Model, reason: The score is 1.00 because the 1st node in retrieval context perfectly contains the entire expected output 'June 2, Monday'!, error: None)
  - ❌ Contextual Relevancy (score: 0.06666666666666667, threshold: 0.7, strict: False, evaluation model: LangChain Model, reason: The score is 0.07 because while the retrieval context does contain the relevant statement 'June 2, Mond

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

OPIK: Started logging traces to the "A.I. Assistant Evaluation 🤖" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=019664b1-8b39-7660-b471-72e6655ef223&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


[TestResult(name='test_case_1', success=False, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=0.5, reason='The score is 0.50 because the inclusion of irrelevant document numbers detracts from the directness of the answer.', strict_mode=False, evaluation_model='LangChain Model', error=None, evaluation_cost=None, verbose_logs='Statements:\n[\n    "Telangana Formation Day is on June 2, 2025",\n    "Document No. 0"\n] \n \nVerdicts:\n[\n    {\n        "verdict": "yes",\n        "reason": null\n    },\n    {\n        "verdict": "no",\n        "reason": "Document number is irrelevant to the date of Telangana Formation Day."\n    }\n]'), MetricData(name='Faithfulness', threshold=0.7, success=True, score=1.0, reason='The score is 1.00 because there are no contradictions, indicating the actual output aligns perfectly with the retrieval context! Great job!', strict_mode=False, evaluation_model='LangChain Model', error=None, evaluation_cost=None, verbose_logs

In [21]:
# dir(results.test_results[0])

In [20]:
test_summary = dict() 
for i in range(len(results.test_results)):
    scores = {metric.name:metric.score for metric in results.test_results[i].metrics_data} 
    test_summary[i] = {
        "query": results.test_results[i].input,
        "actual_output": results.test_results[i].actual_output,
        "expected_output": results.test_results[i].expected_output,
        "actual_context": results.test_results[i].context,
        "retrieval_context": results.test_results[i].retrieval_context,
        "metrics": scores
    }
test_summary

{0: {'query': 'When is Telangana Formation Day in 2025?',
  'actual_output': 'Telangana Formation Day is on June 2, Monday (0).\n',
  'expected_output': 'June 2, Monday',
  'actual_context': ['Holiday Calendar 2025 \n \nJanuary 1, Wednesday \nNew Year’s Day \n \nJanuary 14, Tuesday \nMakara Sankranti \n \nMarch 31, Monday \nIdul Fitr Holiday \n \nMay 1, Thursday \nMay day \n \nJune 2, Monday \nTelangana Formation Day \n \nAugust 15, Friday \nIndependence Day \n \nAugust 27, Wednesday \nVinayaka Chaturthi \n \nOctober 2, Thursday \nGandhi Jayanti / Dusshera \n \nOctober 20, Monday \nDeepavali \n \nDecember 25, Thursday \nChristmas Day \nJanuary \nSu Mo Tu We Th Fr Sa \n   1 2 3 4 \n5 6 7 8 9 10 11 \n12 13 14 15 16 17 18 \n19 20 21 22 23 24 25 \n26 27 28 29 30 31  \n \nFebruary \nSu Mo Tu We Th Fr Sa \n      1 \n2 3 4 5 6 7 8 \n9 10 11 12 13 14 15 \n16 17 18 19 20 21 22 \n23 24 25 26 27 28  \n \nMarch \nSu Mo Tu We Th Fr Sa \n      1 \n2 3 4 5 6 7 8 \n9 10 11 12 13 14 15 \n16 17 18 19 20

In [9]:
scorer.get_summary()

{0: {'query': 'When is Telangana Formation Day in 2025?',
  'actual_output': 'Telangana Formation Day is on June 2, 2025 (Document No. 0).\n',
  'expected_output': 'June 2, Monday',
  'actual_context': ['Holiday Calendar 2025 \n \nJanuary 1, Wednesday \nNew Year’s Day \n \nJanuary 14, Tuesday \nMakara Sankranti \n \nMarch 31, Monday \nIdul Fitr Holiday \n \nMay 1, Thursday \nMay day \n \nJune 2, Monday \nTelangana Formation Day \n \nAugust 15, Friday \nIndependence Day \n \nAugust 27, Wednesday \nVinayaka Chaturthi \n \nOctober 2, Thursday \nGandhi Jayanti / Dusshera \n \nOctober 20, Monday \nDeepavali \n \nDecember 25, Thursday \nChristmas Day \nJanuary \nSu Mo Tu We Th Fr Sa \n   1 2 3 4 \n5 6 7 8 9 10 11 \n12 13 14 15 16 17 18 \n19 20 21 22 23 24 25 \n26 27 28 29 30 31  \n \nFebruary \nSu Mo Tu We Th Fr Sa \n      1 \n2 3 4 5 6 7 8 \n9 10 11 12 13 14 15 \n16 17 18 19 20 21 22 \n23 24 25 26 27 28  \n \nMarch \nSu Mo Tu We Th Fr Sa \n      1 \n2 3 4 5 6 7 8 \n9 10 11 12 13 14 15 \n16 

In [25]:
from ai_eval.services.file import JSONService

json_service = JSONService(
    root_path="/Users/avosseler/Github/Team/ai-assistant-eval/src/notebooks",
    path=".deepeval-cache.json",
)
deepeval_report = json_service.doRead()
deepeval_report

2025-04-22 23:00:39,654 - ai_eval.services.file - INFO - Read: /Users/avosseler/Github/Team/ai-assistant-eval/src/notebooks/.deepeval-cache.json


{'test_cases_lookup_map': {'{"actual_output": "March 14 (Document 0)\\n", "context": ["19 20 21 22 23 24 25 16 17 18 19 20 21 22 21 22 23 24 25 26 27 \\n26 27 28 29 30 31  23 24 25 26 27 28 29 28 29 30 31    \\n       30              \\n \\nOptional Holidays  \\n \\n(Choose only ONE of the 2 holidays mentioned below)  \\nMarch 14, Friday \\nHoli \\n \\nApril 18, Friday \\nGood Friday"], "expected_output": "March 14", "hyperparameters": null, "input": "What date is Holi?", "retrieval_context": ["19 20 21 22 23 24 25 16 17 18 19 20 21 22 21 22 23 24 25 26 27 \\n26 27 28 29 30 31  23 24 25 26 27 28 29 28 29 30 31    \\n       30              \\n \\nOptional Holidays  \\n \\n(Choose only ONE of the 2 holidays mentioned below)  \\nMarch 14, Friday \\nHoli \\n \\nApril 18, Friday \\nGood Friday", "Holiday Calendar 2025 \\n \\nJanuary 1, Wednesday \\nNew Year\\u2019s Day \\n \\nJanuary 14, Tuesday \\nMakara Sankranti \\n \\nMarch 31, Monday \\nIdul Fitr Holiday \\n \\nMay 1, Thursday \\nMay d

In [ ]:
dataset = [
    {"input": "What is the capital of France?", "expected_output": "Paris", "context": "France's capital is Paris."},
    # ... more items
]

from opik import Experiment

experiment = Experiment(name="DeepEval Integration Test")
for item in dataset:
    # Assume you have DeepEval results in `result`
    experiment.log(
        input=item["input"],
        output=your_llm_output,
        expected_output=item["expected_output"],
        metrics={"accuracy": result.score},  # Add other metrics as needed
        context=item.get("context", "")
    )
experiment.end()


In [16]:
from langchain_google_vertexai.model_garden import ChatAnthropicVertex
from ai_eval.config import global_config as glob

llm = ChatAnthropicVertex(
    model_name="publishers/anthropic/models/claude-3-sonnet@20240229",
    project=glob.GCP_PROJECT,
    #location="us-central1"
)

response = llm.invoke("Explain quantum computing in simple terms")
print(response.content)

Retrying langchain_google_vertexai.model_garden.ChatAnthropicVertex._generate.<locals>._completion_with_retry_inner in 4.0 seconds as it raised NotFoundError: Error code: 404.
Retrying langchain_google_vertexai.model_garden.ChatAnthropicVertex._generate.<locals>._completion_with_retry_inner in 4.0 seconds as it raised NotFoundError: Error code: 404.


NotFoundError: Error code: 404

# RAGAS

In [7]:
config = {
    "model": "gemini-2.0-flash-001",  # or other model IDs
    "temperature": 0.4,
    "max_tokens": None,
    "top_p": 0.8,
    # For Vertex AI only:
    "project": glob.GCP_PROJECT,  # Required for Vertex AI
    "location": "europe-west3",     # Required for Vertex AI
}

In [9]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_google_vertexai import ChatVertexAI
from ai_eval.resources.get_models import InitModels

models = InitModels()

# Or initialize with Vertex AI
evaluator_llm = LangchainLLMWrapper(models.eval_model)

2025-04-14 22:34:49,671 - ai_eval.resources.get_models - INFO - Fetching ollama models: ['qa_generator', 'rag_generator', 'eval_model']
2025-04-14 22:34:49,759 - ai_eval.resources.get_models - INFO - Models loaded successfully
2025-04-14 22:34:49,759 - ai_eval.resources.get_models - INFO - QA Generator: model='llama3.2' temperature=0.3
2025-04-14 22:34:49,759 - ai_eval.resources.get_models - INFO - RAG Generator: model='llama3.2' temperature=0.1
2025-04-14 22:34:49,760 - ai_eval.resources.get_models - INFO - Evaluation Model: model='mistral:7b' temperature=0.3


In [10]:
from langchain_google_vertexai import HarmCategory, HarmBlockThreshold

safety_settings = {
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    # Add other safety settings as needed
}

evaluator_llm = LangchainLLMWrapper(models.eval_model,
                                    #safety_settings=safety_settings
                                    )

In [11]:
# Vertex AI Embeddings
from langchain_google_vertexai import VertexAIEmbeddings

evaluator_embeddings = LangchainEmbeddingsWrapper(VertexAIEmbeddings(
    model_name="text-embedding-004",  # or other available model
    project=config["project"],  # Your GCP project ID
    location=config["location"]  # Your GCP location
))

In [13]:
from ragas import SingleTurnSample
from ragas.metrics import AspectCritic

test_data = {
    "user_input": "summarise given text\nThe company reported an 8% rise in Q3 2024, driven by strong performance in the Asian market. Sales in this region have significantly contributed to the overall growth. Analysts attribute this success to strategic marketing and product localization. The positive trend in the Asian market is expected to continue into the next quarter.",
    "response": "The company experienced an 8% increase in Q3 2024, largely due to effective marketing strategies and product adaptation, with expectations of continued growth in the coming quarter.",
}

metric = AspectCritic(name="summary_accuracy",llm=evaluator_llm, definition="Verify if the summary is accurate.")
test_data = SingleTurnSample(**test_data)
await metric.single_turn_ascore(test_data)

1